In [ ]:
%%capture

import warnings

warnings.filterwarnings("ignore")

import _section1_utils as section1
import _section3_utils as section3
import calitp_data_analysis.magics
import great_tables as gt
import pandas as pd
from IPython.display import HTML, Image, Markdown, display, display_html
from shared_utils import catalog_utils, rt_dates, rt_utils

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
# Comment out and leave this cell right below pandas
# organization_name = "City of Lawndale"

In [ ]:
%%capture_parameters
organization_name

In [ ]:
most_recent_date = pd.to_datetime(rt_dates.DATES["mar2024"])
year = most_recent_date.year

In [ ]:
# Testing certain operators
# organization_name = "Peninsula Corridor Joint Powers Board"
# organization_name = "City and County of San Francisco"

In [ ]:
# Load first dataset
operator_profiles = section1.load_operator_profiles(organization_name)

In [ ]:
# Need to load "name" using this function rather than taking it from
# operator profiles above because it differs.
name = section1.organization_name_crosswalk(organization_name)

In [ ]:
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

In [ ]:
# Datasets for Section 1
operator_route_map = section1.load_operator_map(name)
monthly_scheduled_service = section1.summarize_monthly(name)

In [ ]:
# Datasets for Section 2
sched_vp_df = section3.load_schedule_vp_metrics(organization_name)
gtfs_schedule_keys = list(sched_vp_df.schedule_gtfs_dataset_key.unique())

In [ ]:
# Set drop down menu to be on the upper right
display(
    HTML(
        """
<style>
form.vega-bindings {
  position: absolute;
  right: 0px;
  top: 0px;
}
</style>
"""
    )
)

# {organization_name}

In [ ]:
display(
    Markdown(
        f"""<h2 style="border-bottom: 3px solid #8CBCCB;"><b>Operator Overview</b></h2>
        """
    )
)

In [ ]:
try:
    date = operator_profiles["Date"].dt.to_pydatetime()[0]
except:
    pass

In [ ]:
try:
    display(
        Markdown(
            f"""This section presents an overview of statistics for {organization_name}, using data from the most recent date of
      {date.month}/{date.year}."""
        )
    )
except:
    pass

In [ ]:
try:
    display(
        Markdown(
            f"""{organization_name} ran <b>{operator_profiles['# routes'].values[0]}</b> 
        unique routes. Below is the breakdown of the routes."""
        )
    )
except:
    display(Markdown(f"""{organization_name} doesn't have an operator profile."""))

In [ ]:
try:
    route_typology_df = section1.route_typology(operator_profiles)
except:
    display(
        Markdown(f"""{organization_name} doesn't have route typologies available.""")
    )

In [ ]:
if route_typology_df.total_routes.sum() == 0:
    display(
        Markdown(f"""{organization_name} doesn't have route typologies available.""")
    )
else:
    display(
        section1.basic_pie_chart(
            route_typology_df,
            "route_type",
            "total_routes",
            "Total Routes by Typology",
        )
    )

In [ ]:
try:
    display(
        Markdown(
            f"""{organization_name} provided <b>{int(operator_profiles['service miles'].values[0])}</b> miles of public transit.
        The average length of a route is <b>{int(operator_route_map['service miles'].mean())}</b> miles.
        """
        )
    )
except:
    pass

In [ ]:
try:
    display(
        section1.basic_bar_chart(
            section1.shortest_longest_route(operator_route_map),
            "service miles",
            "Route",
            f"Longest and Shortest Route",
        )
    )
except:
    pass

In [ ]:
display(
    Markdown(f"{organization_name} provides public transit in the following counties:")
)

In [ ]:
try:
    section1.counties_served(operator_route_map)
except:
    display(Markdown(f"""{organization_name} doesn't have an route geographies."""))

In [ ]:
try:
    display(
        Markdown(
            f"""{organization_name} stopped at <b>{int(operator_profiles['# stops'].values[0])}</b> unique stops, 
            totaling to <b>{int(operator_profiles['# arrivals'].values[0])}</b> arrivals. The average arrivals per stop 
            was <b>{int(operator_profiles['avg arrivals per stop'].values[0])}</b>.
        """
        )
    )
except:
    pass

In [ ]:
display(
    Markdown(f"Use the dropdown menu on the right to select a date to view Total Scheduled Service Hours by month.")
)

In [ ]:
monthly_scheduled_service_df_agg = section1.summarize_monthly(name)

In [ ]:
section1.single_bar_chart_dropdown(
    monthly_scheduled_service_df_agg,
    "day of week",
    "total service (hours)",
    "time of day",
    "Total Scheduled Service Hours",
    "full_date",
)

In [ ]:
display(
    Markdown(
        f"""<h2 style="border-bottom: 3px solid #8CBCCB;"><b>Detailed Route-Direction Overview</b></h2><br>
               
        """
    )
)

In [ ]:
section3.trips_by_gtfs(sched_vp_df)

In [ ]:
display(
    Markdown(
        f"""Statistics presented on the <b>route</b> level. Scroll down to filter for a specific route. 
           """
    )
)

In [ ]:
section3.filtered_route(sched_vp_df)